In [20]:
import pandas as pd
import csv
from datetime import datetime
from datetime import timedelta

#Read Effective and final file merged them 

df=pd.read_csv('Effective.csv')
df1=pd.read_csv('Final.csv')

df=df.drop(['Final Date'],axis=1)
df=df.rename(columns={'Effective Date':'Effective/Final Date'})
df1=df1.drop(['Effective Date'],axis=1)
df1=df1.rename(columns={'Final Date':'Effective/Final Date'})

#merging of Two files
df_final=pd.concat([df,df1])

df_final['Document Number']=df_final['Document Number'] + ".pdf"
df_final['Effective/Final Date']=pd.to_datetime(df_final['Effective/Final Date'])
df_final['Effective/Final Date'] = df_final['Effective/Final Date'].dt.strftime('%m/%d/%y')


#move document names to txt file

x = ""
n = len(df_final['Document Number'])
import math
r = math.ceil(n/400)
for i in range(r):
    x = ""
    for p in range(400*i,400*(i+1)):
        if p < len(df_final['Document Number']):
            x += " "
            x += df_final.iloc[p]['Document Number']
        else: break
                
    file1 = open(f'SCS_E&F_{i+1}.txt', 'w')
    file1.write("dir /s /t:w ")
    file1.write(x)
    time = datetime.today()
    time = time.strftime("%d_%B")
    path = f">D:\check\\SCS_{time}_{i+1}.txt"
    file1.write(path)
    file1.close()
    

#Export result to CSV file i.e CDOCS_Report

print(len(df_final))
today = datetime.today()
yest = today - timedelta(days = 1)
yest = yest.strftime("%d_%B")

Cdoc_report=f"CDOCS_report_{yest}.csv"
df_final.to_csv(Cdoc_report,index=False)


#Read obsolete file

df_obsolete=pd.read_csv('Obsolete Documents.csv')
df_obsolete['Document Number']=df_obsolete['Document Number'] + ".pdf"
df_obsolete['Comments']='Moved out of SCS'
   #print(df_obsolete)
obsolete_report=f"Obsolete_report_{yest}.csv"
df_obsolete.to_csv(obsolete_report,index=False)    
    


#move document to txt file

y= ""
for i in df_obsolete['Document Number']:
    y+=" "
    y+=i
    
file2= open('SCS_Obs.txt','w')
file2.write("dir /s /t:w ")
file2.write(y)
file2.close()






649


In [21]:
#Processing on the SCS file

import numpy as np

df_temp=pd.read_csv('SCS_temp.csv',encoding= 'unicode_escape',header=None)
df_temp = df_temp[df_temp[0].str.contains("pdf")]
x = df_temp[0].str.split(' ')

date  =[]
doc = []
for i in x:
    date.append(i[0])
    doc.append(i[-1])
    
df_temp['Modified_date']=pd.to_datetime(date)
df_temp['Modified_date'] = df_temp['Modified_date'].dt.strftime('%m/%d/%y')
#df_temp['Modified_date'] = datetime.strptime(df_temp['Modified_date'], '%m %d %y').date()
df_temp['Document Number']=doc

df_temp.drop(df_temp.columns[0],axis=1,inplace =True)
#print(df_temp)
print(len(df_temp))
#Exporting result to csv i.e SCS Report
df_temp.to_csv('SCS_FINAL.csv',index=False)


648


In [22]:
#Checking or validation of both report for Effective date and count
df_CDOCS = pd.read_csv(Cdoc_report)
df_SCS = pd.read_csv('SCS_FINAL.csv')
#print(len(df_SCS))


inner_join = pd.merge(df_CDOCS, 
                      df_SCS, 
                      on ='Document Number', 
                      how ='right')
#print(inner_join)
df_new=pd.DataFrame()
#inner_join['check']=np.where(inner_join['Modified_date'] >= inner_join['Effective/Final Date'],'True', 'False')
inner_join['check']=pd.to_datetime(inner_join['Modified_date']) >= pd.to_datetime(inner_join['Effective/Final Date'])
df_new['Document Number']=inner_join['Document Number']
df_new['Modified_date']=inner_join['Modified_date']
df_new['Effective/Final Date']=inner_join['Effective/Final Date']
df_new['check']=inner_join['check']
print(len(df_new))

#Export all validation report to SCS_Final Report
Scs_report=f"SCS_report_{yest}.csv"
df_new.to_csv(Scs_report,index=False);




648


In [23]:
#Check the count of document from both the report(CDOCS and SCS)

df_CDOCS = pd.read_csv(Cdoc_report)
df_SCS = pd.read_csv(Scs_report)

count1=len(df_CDOCS['Document Number'])
count2=len(df_SCS['Document Number'])

if count1==count2:
    print('All documents are mapped')
else:
    print('Below documents are not mapped')
    
    left_join = pd.merge(df_CDOCS, 
                      df_SCS, 
                      on ='Document Number', 
                      how ='left')  
    #print(left_join)
    doc=[]
    for i,j in zip(left_join['Modified_date'],left_join['Document Number']):
        if pd.isna(i):
            #result['Modified_date']=i
            #result['Document Number']=j
            doc.append(j)
    print(doc)
    

Below documents are not mapped
['REC-641793.pdf']


In [25]:
#Gives the list of not updated document on SCS

df_SCS = pd.read_csv(Scs_report)
#print(df_SCS)
doc1 = []
for i,j in zip(df_SCS['check'],df_SCS['Document Number']):
    if i == False:
        doc1.append(j)
print('Count of non updated document',len(doc1))    
print('list of non updated document')
print(doc1)

Count of non updated document 0
list of non updated document
[]


In [19]:
import os
import shutil
from datetime import datetime
from datetime import timedelta

if os.path.isfile(r"Effective.csv"):
          os.remove(r"Effective.csv")
if os.path.isfile(r"Final.csv"):
          os.remove(r"Final.csv")
if os.path.isfile(r"Obsolete Documents.csv"):
          os.remove(r"Obsolete Documents.csv")
if os.path.isfile(r"SCS_Obs.txt"):
          os.remove(r"SCS_Obs.txt")
if os.path.isfile(r"SCS_FINAL.csv"):
          os.remove(r"SCS_FINAL.csv")
        
if os.path.isfile(r"SCS_E&F_1.txt"):
          os.remove(r"SCS_E&F_1.txt")
if os.path.isfile(r"SCS_E&F_2.txt"):
          os.remove(r"SCS_E&F_2.txt")
if os.path.isfile(r"SCS_E&F_3.txt"):
          os.remove(r"SCS_E&F_3.txt")
if os.path.isfile(r"SCS_E&F_4.txt"):
         os.remove(r"SCS_E&F_4.txt")
if os.path.isfile(r"SCS_E&F_5.txt"):
         os.remove(r"SCS_E&F_5.txt")

    
today = datetime.today() 
yest = today - timedelta(days = 1)
yest = yest.strftime("%d_%B")
today=today.strftime("%d_%B")
directory =f"{today}"
parent_dir = r"D:\Users\snyayadh\OneDrive - Amgen\Automation"
path=os.path.join(parent_dir, directory) 
os.mkdir(path)

cdocs_report=f"CDOCS_report_{yest}.csv"
#cdocs_report="CDOCS_report_03_January.csv"
path1=os.path.join(parent_dir,cdocs_report)
path2=f"{path}\{cdocs_report}"
shutil.move(path1, path2)

scs_report=f"SCS_report_{yest}.csv"
#scs_report="SCS_report_03_January.csv"
path1=os.path.join(parent_dir,scs_report)
path2=f"{path}\{scs_report}"

shutil.move(path1, path2)


obs_report=f"Obsolete_report_{yest}.csv"
#obs_report="Obsolete_report_03_January.csv"
path1=os.path.join(parent_dir,obs_report)
path2=f"{path}\{obs_report}"

shutil.move(path1, path2)
print("Done")



Done
